# Модель выбытия жилищного фонда 

In [13]:
import requests
from cmasf import pandas_sql as pds
import pandas as pd
import sqlalchemy as sa
import re
from os import path, remove
import glob
from bs4 import BeautifulSoup
import datetime as dt
import numpy as np

strDB_Temp=path.join('..','Temp') # путь к папке с временными файлами
strHousesDBPath=path.join('..','DB', 'Houses.sqlite3') # путь к рабочей базе данных SQLite Houses
strBaseDB_URL=r'https://www.reformagkh.ru/opendata?gid=2208161&cids=house_management&page={page_num}&pageSize={page_size}'

strExportDataURL=r'https://www.reformagkh.ru/opendata/export/{num}'

## База данных - сбор

<p>Источник данных - https://www.reformagkh.ru/opendata?gid=2208161&cids=house_management&page=1&pageSize=100</p>
<p>Качаем паспорта домов из <b>"Реестра домов"</b> по каждому региону</i> </p>

<div class="alert alert-block alert-info">
<p>Условие попадания дома в расчеты модели: </p>
    <ul>
        <li>ненулевые данные об общей площади жилых помещений <b>(living_rooms_sq > 0)</b></li>
        <li>ненулевое количество жилых помещений <b>(living_rooms_amount > 0)</b></li>
        <li>соотношение этих параметров находилось в интервале между 3 и 1000 <b>(3 < living_rooms_sq/living_rooms_amount < 1000 )</b></li>
    </ul>
</div>

<div class="alert alert-block alert-success">
<p><b>Поля, используемые в расчетах:</b></p>
     <ol>
         <li>Возраст здания <b>(текущий год - commission_year)</b></li>
         <li>Этажность</li>
      </ul>
</div>


In [2]:
strHouses_table='houses'
strHouses_data_pass='houses_columns'
strInfoTable='update_info' 

In [3]:
re_a_export=re.compile(r'/opendata/export/\d+') # объект для поиска признака ссылки на файл csv
re_p_date=re.compile('Актуальная версия:') # объект для поиска даты актиализации

re_cap=re.compile('\s(по [А-Яа-я -]+)') # объект для поиска части заголовка - "по Алтайскому краю" - для идентификации ссылки на файл

re_date=re.compile('\d{2}\.\d{2}\.\d{4}') # объект для поиска захвата даты актуализации

re_pass_dt=re.compile('p-5 mx-4') # объект для поиска куска с паспортом набора даных

def delete_temp_files(folder=strDB_Temp):
    """Удаление файлов во временой папке - ОСТОРОЖНО!!! УДАЛЯЕТ ВСЕ ФАЙЛЫ В УКАЗАННОЙ ПАПКЕ!!!
    Вызывать для очистки папки временных файлов после обновления базы данных"""
    files = glob.glob(path.join(folder, '*.zip'))
    for f in files:
        remove(f)

def read_html_page(page_num=1, page_size=100):
    
    dom_html=requests.get(strBaseDB_URL.format(page_num=page_num, page_size=page_size))
    dsp=BeautifulSoup(dom_html.text, 'html.parser')

    p_caps=dsp.findAll('p', text=re.compile('Реестр домов по'))
    if not p_caps: 
#         print('not data')
        return None
    lst_data=list()
    for p in p_caps:
        a_exp=p.find_next('div', class_='row').find('a', href=re_a_export)
        p_date=p.find_next('p', text=re_p_date)
        lst_data.append({'name':re_cap.search(p.text).group(0).strip(),
                        'date':re_date.search(p_date.text).group(0), 
                        'link':r'https://www.reformagkh.ru{suff}'.format(suff=a_exp['href'])})
    return pd.DataFrame(lst_data)

def read_db_password():
    dom_html=requests.get(strBaseDB_URL.format(page_num=1, page_size=10))
    dsp=BeautifulSoup(dom_html.text, 'html.parser')

    p_cap=dsp.find('p', text=re.compile('Реестр домов по'))
    
    div_pass=p_cap.find_next('div', class_=re_pass_dt).findAll('div', class_='row')
    lst_pass=list()
    for d in div_pass:
        dd=d.findAll('div')
        lst_pass.append({'name':dd[0].text, 'description':dd[1].text})
    return lst_pass

In [4]:
pdfDBFiles=pd.concat([read_html_page(page_num=page) for page in range(1, 3)])
print(pdfDBFiles)

lst_fileds=read_db_password()

                                     name        date  \
0                      по Алтайскому краю  01.08.2020   
1                     по Амурской области  01.08.2020   
2                по Архангельской области  01.08.2020   
3                 по Астраханской области  01.08.2020   
4                 по Белгородской области  01.08.2020   
..                                    ...         ...   
80                по Чеченской Республике  01.08.2020   
81                по Чувашской Республике  01.08.2020   
82       по Чукотскому автономному округу  01.08.2020   
83  по Ямало-Ненецкому автономному округу  01.08.2020   
84                 по Ярославской области  01.08.2020   

                                             link  
0   https://www.reformagkh.ru/opendata/export/118  
1   https://www.reformagkh.ru/opendata/export/167  
2   https://www.reformagkh.ru/opendata/export/171  
3    https://www.reformagkh.ru/opendata/export/91  
4   https://www.reformagkh.ru/opendata/export/170  
.. 

In [5]:
for k, v in pdfDBFiles.iterrows():
    print('read for ', v['name'], v['link'], end='...')
    r = requests.get(v['link'])
    with open(path.join(strDB_Temp, '{name}.zip'.format(name=v['name'])), 'wb') as fz_tmp:
        fz_tmp.write(r.content)
    print('done')

read for  по Алтайскому краю https://www.reformagkh.ru/opendata/export/118...done
read for  по Амурской области https://www.reformagkh.ru/opendata/export/167...done
read for  по Архангельской области https://www.reformagkh.ru/opendata/export/171...done
read for  по Астраханской области https://www.reformagkh.ru/opendata/export/91...done
read for  по Белгородской области https://www.reformagkh.ru/opendata/export/170...done
read for  по Брянской области https://www.reformagkh.ru/opendata/export/104...done
read for  по Владимирской области https://www.reformagkh.ru/opendata/export/103...done
read for  по Волгоградской области https://www.reformagkh.ru/opendata/export/90...done
read for  по Вологодской области https://www.reformagkh.ru/opendata/export/169...done
read for  по Воронежской области https://www.reformagkh.ru/opendata/export/102...done
read for  по городу Москве https://www.reformagkh.ru/opendata/export/101...done
read for  по городу Санкт-Петербургу https://www.reformagkh.ru/op

## Собираем данные из ZIP-файлов в один датафрейм

In [6]:
lst_fileds=read_db_password()
pdDataPass=pd.DataFrame(lst_fileds)

In [7]:
# lst_excluded_fields=['region_id', 'area_id', 'city_id', 'street_id', 'shortname_region', 'formalname_region',
#                      'shortname_area', 'formalname_area', 'shortname_city', 'formalname_city', 'shortname_street',
#                      'formalname_street', 'house_number', 'building', 'block', 'letter', 'management_organization_id',
#                      'method_of_forming_overhaul_fund', 'area_land', 'parking_square', 'playground', 'sportsground',
#                      'other_beautification', 'chute_count', 'electrical_entries_count', 'sewerage_cesspools_volume']

In [8]:
lst_included_fields=['houseguid','built_year', 'house_type', 'is_alarm', 'floor_count_max', 'elevators_count','area_total',
                     'area_residential', 'foundation_type', 'floor_type', 'wall_material', 'heating_type',
                     'hot_water_type', 'cold_water_type', 'living_quarters_count']

In [9]:
zip_files=glob.glob(path.join(strDB_Temp, '*.zip'))

lstDTF=list()
for zf in zip_files:
    print('read ', zf, end='...')
    pdf=pd.read_csv(zf, sep=';')
    pdf.replace(['Не заполнено','Нет данных','нет данных'], np.nan, inplace=True) #заменяем на NaN
#     pdf=pdf[pdf.columns.difference(lst_excluded_fields)].set_index('houseguid')
    pdf=pdf[lst_included_fields].set_index('houseguid') # включаем только выбранные колонки
    lstDTF.append(pdf)
    print('done for', pdf.shape)
    
pdfAll=pd.concat(lstDTF)
# pdfAll=pdfAll.replace(dict.fromkeys(['Не заполнено','Нет данных'], np.nan))
pdfAll

read  Temp\по Алтайскому краю.zip...done for (10050, 14)
read  Temp\по Амурской области.zip...done for (10162, 14)
read  Temp\по Архангельской области.zip...done for (21688, 14)
read  Temp\по Астраханской области.zip...done for (6033, 14)
read  Temp\по Белгородской области.zip...done for (6633, 14)
read  Temp\по Брянской области.zip...done for (7414, 14)
read  Temp\по Владимирской области.zip...done for (14052, 14)
read  Temp\по Волгоградской области.zip...done for (11843, 14)
read  Temp\по Вологодской области.zip...

e:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


done for (16868, 14)
read  Temp\по Воронежской области.zip...done for (10583, 14)
read  Temp\по городу Москве.zip...done for (34418, 14)
read  Temp\по городу Санкт-Петербургу.zip...done for (27747, 14)
read  Temp\по городу Севастополю.zip...done for (4066, 14)
read  Temp\по Еврейской автономной области.zip...done for (2078, 14)
read  Temp\по Забайкальскому краю.zip...done for (8110, 14)
read  Temp\по Ивановской области.zip...done for (10807, 14)
read  Temp\по Иркутской области.zip...done for (24241, 14)
read  Temp\по Кабардино-Балкарской Республике.zip...done for (2535, 14)
read  Temp\по Калининградской области.zip...done for (15359, 14)
read  Temp\по Калужской области.zip...done for (9392, 14)
read  Temp\по Камчатскому краю.zip...done for (4007, 14)
read  Temp\по Карачаево-Черкесской Республике.zip...done for (1380, 14)
read  Temp\по Кемеровской области.zip...done for (20755, 14)
read  Temp\по Кировской области.zip...

e:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


done for (17309, 14)
read  Temp\по Костромской области.zip...done for (8797, 14)
read  Temp\по Краснодарскому краю.zip...done for (20947, 14)
read  Temp\по Красноярскому краю.zip...done for (19272, 14)
read  Temp\по Курганской области.zip...done for (3839, 14)
read  Temp\по Курской области.zip...done for (5393, 14)
read  Temp\по Ленинградской области.zip...done for (18195, 14)
read  Temp\по Липецкой области.zip...done for (7189, 14)
read  Temp\по Магаданской области.zip...done for (1963, 14)
read  Temp\по Московской области.zip...done for (56845, 14)
read  Temp\по Мурманской области.zip...done for (7859, 14)
read  Temp\по Ненецкому автономному округу.zip...done for (765, 14)
read  Temp\по Нижегородской области.zip...done for (32449, 14)
read  Temp\по Новгородской области.zip...done for (5630, 14)
read  Temp\по Новосибирской области.zip...done for (15113, 14)
read  Temp\по Омской области.zip...done for (11389, 14)
read  Temp\по Оренбургской области.zip...done for (13243, 14)
read  Temp\

e:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,7,8,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


done for (16507, 14)
read  Temp\по Сахалинской области.zip...done for (12471, 14)
read  Temp\по Свердловской области.zip...done for (42879, 14)
read  Temp\по Смоленской области.zip...done for (7635, 14)
read  Temp\по Ставропольскому краю.zip...done for (9566, 14)
read  Temp\по Тамбовской области.zip...done for (6277, 14)
read  Temp\по Тверской области.zip...done for (11796, 14)
read  Temp\по Томской области.zip...done for (7458, 14)
read  Temp\по Тульской области.zip...done for (19040, 14)
read  Temp\по Тюменской области.zip...done for (9486, 14)
read  Temp\по Удмуртской Республике.zip...done for (9571, 14)
read  Temp\по Ульяновской области.zip...done for (7817, 14)
read  Temp\по Хабаровскому краю.zip...done for (10068, 14)
read  Temp\по Ханты-Мансийскому автономному округу.zip...done for (14930, 14)
read  Temp\по Челябинской области.zip...done for (25138, 14)
read  Temp\по Чеченской Республике.zip...done for (2978, 14)
read  Temp\по Чувашской Республике.zip...done for (6593, 14)
read 

,built_year,house_type,is_alarm,floor_count_max,elevators_count,area_total,area_residential,foundation_type,floor_type,wall_material,heating_type,hot_water_type,cold_water_type,living_quarters_count
houseguid,,,,,,,,,,,,,,
fb1c6015-17bf-4e6a-975e-e539769e4648,1934.0,Многоквартирный дом,Нет,2.0,0.0,"579,10","504,30",Ленточный,Деревянные,Деревянные,Центральное,Открытая с отбором сетевой воды на горячее вод...,Центральное,14.0
16acd9c1-2e40-4400-8ba5-2f3733c4c82c,1964.0,Многоквартирный дом,Нет,2.0,0.0,"424,00","379,30",Ленточный,Деревянные,Кирпич,Центральное,Отсутствует,Центральное,8.0
da846414-4337-38e4-846b-31f1c7200c07,1993.0,Многоквартирный дом,Нет,3.0,0.0,"1376,20","1238,50",Ленточный,Железобетонные,Кирпич,Центральное,Отсутствует,Центральное,22.0
31e0df72-9b33-2afc-7b6e-716b9dcebe91,1995.0,Многоквартирный дом,Нет,3.0,0.0,"1286,90","1286,90",Ленточный,Железобетонные,Панельные,Центральное,Отсутствует,Центральное,27.0
aefbf6f8-88ae-70eb-d348-8f0ce78ae613,NaN,Жилой дом блокированной застройки,Нет,1.0,0.0,"42,70","25,70",NaN,Деревянные,Иные,Печное,Отсутствует,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6e9a01e2-f945-4698-9440-11b03a79fa87,NaN,NaN,Нет,NaN,NaN,"149,20","149,20",NaN,NaN,NaN,NaN,NaN,NaN,NaN
96813a20-b6a3-4623-aeef-81e73c91cb14,NaN,NaN,Нет,NaN,NaN,"134,40","134,40",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1405946b-7a94-4a79-b80b-ceaba81e26b7,NaN,Многоквартирный дом,Нет,2.0,0.0,"145,10","145,10",NaN,Железобетонные,Панельные,Центральное,Отсутствует,Центральное,NaN


Замена запятых на точки:

In [10]:
# Код для широкого датафрейма (когда используем lst_excluded_fields) в предыдущем блоке

# pdfAll[['area_common_property',
#         'area_non_residential',
#         'area_non_residential',
#         'area_residential',
#         'area_total',
#         'basement_area']] = pdfAll[['area_common_property',
#                                     'area_non_residential',
#                                     'area_non_residential',
#                                     'area_residential',
#                                     'area_total',
#                                     'basement_area']].applymap(lambda x: float(x.replace(',', '.')) if(pd.notnull(x)) else x)


In [11]:
pdfAll[['area_residential',
        'area_total']] = pdfAll[['area_residential',
                                 'area_total']].applymap(lambda x: float(x.replace(',', '.')) if(pd.notnull(x)) else x)

In [12]:

# pdfAll[['built_year']]=pdfAll[['built_year']].applymap(lambda x: x.astype(int) if(pd.notnull(x)) else x)


In [200]:
# Для working dataframe отбираем данные без пропусков даты постройки дома и накладываем ограничение [1800;2020]
wdf=pdfAll.copy()
wdf = wdf[wdf['built_year'].notna()].sort_values(by=['built_year'])
# pdfAll['built_year'] = pdfAll['built_year'].astype(int) # Преобразование столбца 'built_year' в целочисленные значения
wdf = wdf[(wdf['built_year'] >= 1800) & (wdf['built_year'] <= 2020)]
wdf['age']=2020-wdf['built_year']
wdf

## Запись в БД

In [14]:
conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strHousesDBPath))

# pdfSQL = pds.DataFrameDATA(pdfAll)

pdfAll.to_sql(strHouses_table, if_exists='replace', con=conWork)
pdDataPass.to_sql(strHouses_data_pass, if_exists='replace', con=conWork)

strCreateQ='''CREATE TABLE IF NOT EXISTS {tab_name} ("utable" TEXT, "udate" TEXT NOT NULL, PRIMARY KEY("utable"))'''
conWork.execute(strCreateQ.format(tab_name=strInfoTable))

_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
                                             update_date=dt.datetime.now().strftime('%Y-%m-%d'), 
                                             table_name=strHouses_table))

In [32]:
# delete_temp_files()

## Ненужное:

In [18]:
pdfSQL.loc[pdfSQL['floor_count_max'].between(1, 5) ].groupby(by=['floor_count_max', 'wall_material', 'built_year']).agg( {'area_residential' : np.nansum, 
                                                                                                                          'living_quarters_count':np.nanmean } )

# , 'area_residential':'sum', 'living_quarters_count':'np.mean'

area_residential  \
floor_count_max wall_material                           built_year                     
1.0             Блочные                                 1858.0                  81.9   
                                                        1882.0                 127.2   
                                                        1890.0                 189.7   
                                                        1892.0                 135.6   
                                                        1896.0                 479.9   
...                                                                              ...   
5.0             Смешанные                               2017.0               32321.0   
                                                        2018.0                1510.4   
                Шлакобетон (блоки)                      1981.0                2654.5   
                                                        2018.0                3364.0   
                Шлакокерамзитобетонная 1-слойная панель 1973.0                1734.4   

                                                                    living_quarters_count  
floor_count_max wall_material                           built_year                         
1.0             Блочные                                 1858.0                   4.000000  
                                                        1882.0                   5.000000  
                                                        1890.0                   2.000000  
                                                        1892.0                   3.000000  
                                                        1896.0                   6.500000  
...                                                                                   ...  
5.0             Смешанные                               2017.0                 107.285714  
                                                        2018.0                  27.000000  
                Шлакобетон (блоки)                      1981.0                  90.000000  
                                                        2018.0                  75.000000  
                Шлакокерамзитобетонная 1-слойная панель 1973.0                  60.000000  

[4132 rows x 2 columns]

In [17]:
pdfSQL.loc[pdfSQL['wall_material'].str.contains('бетон', case=False) & (pdfSQL['floor_count_max']==1), ('address', 'wall_material', 'living_quarters_count')].sort_values(by='living_quarters_count')

,address,wall_material,living_quarters_count
houseguid,,,
14941dcc-f248-4d5f-9a0f-45d4efee4fc1,"г. Москва, ул. Головачева, д. 2, стр. инв11",Железобетон,23.0
4f86af32-5a9c-4c4e-aa16-8ded016561cb,"г. Санкт-Петербург, пр-кт. Советский (Усть-Сла...",Железобетон,192.0
dbaa5c03-0777-410e-8859-0f348786ccf8,"обл. Московская, г. Реутов, ул. им академика В...",Керамзитобетонная 1-слойная панель,264.0
9b71d047-ba91-41cd-ab28-127edd581131,"обл. Московская, г. Лыткарино, мкр. 4а, д. 7",Железобетонная панель,306.0
